# Exercici 1

Crea almenys tres models de classificació diferents per intentar predir el millor possible l’endarreriment dels vols (ArrDelay) de DelayedFlights.csv. Considera si el vol ha arribat tard o no (ArrDelay > 0).

In [411]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics 
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import numpy as np

In [2]:
df=pd.read_csv('DelayedFlights.csv')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df.head()

,Unnamed: 0,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN
1,1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
2,2,2008,1,3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN
3,4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0
4,5,2008,1,3,4,1940.0,1915,2121.0,2110,WN,378,N726SW,101.0,115.0,87.0,11.0,25.0,IND,JAX,688,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN


In [5]:
#elimino la columna contador de filas Unnamed: 0
df = df.drop('Unnamed: 0', 1)

In [6]:
df.describe().round(3)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,1936758.0,1936758.000,1936758.000,1936758.000,1936758.000,1936758.000,1929648.000,1936758.000,1936758.000,1928371.000,1936560.000,1928371.000,1928371.000,1936758.000,1936758.000,1929648.000,1936303.000,1936758.000,1936758.000,1247488.000,1247488.000,1247488.000,1247488.000,1247488.000
mean,2008.0,6.111,15.753,3.985,1518.534,1467.473,1610.141,1634.225,2184.263,133.306,134.303,108.277,42.200,43.185,765.686,6.813,18.232,0.000,0.004,19.179,3.704,15.022,0.090,25.296
std,0.0,3.483,8.776,1.996,450.485,424.767,548.178,464.635,1944.702,72.060,71.341,68.643,56.785,53.403,574.480,5.274,14.339,0.018,0.063,43.546,21.493,33.833,2.023,42.055
min,2008.0,1.000,1.000,1.000,1.000,0.000,1.000,0.000,1.000,14.000,-25.000,0.000,-109.000,6.000,11.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,2008.0,3.000,8.000,2.000,1203.000,1135.000,1316.000,1325.000,610.000,80.000,82.000,58.000,9.000,12.000,338.000,4.000,10.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
50%,2008.0,6.000,16.000,4.000,1545.000,1510.000,1715.000,1705.000,1543.000,116.000,116.000,90.000,24.000,24.000,606.000,6.000,14.000,0.000,0.000,2.000,0.000,2.000,0.000,8.000
75%,2008.0,9.000,23.000,6.000,1900.000,1815.000,2030.000,2014.000,3422.000,165.000,165.000,137.000,56.000,53.000,998.000,8.000,21.000,0.000,0.000,21.000,0.000,15.000,0.000,33.000
max,2008.0,12.000,31.000,7.000,2400.000,2359.000,2400.000,2400.000,9742.000,1114.000,660.000,1091.000,2461.000,2467.000,4962.000,240.000,422.000,1.000,1.000,2436.000,1352.000,1357.000,392.000,1316.000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1936758 entries, 0 to 1936757
Data columns (total 29 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Year               int64  
 1   Month              int64  
 2   DayofMonth         int64  
 3   DayOfWeek          int64  
 4   DepTime            float64
 5   CRSDepTime         int64  
 6   ArrTime            float64
 7   CRSArrTime         int64  
 8   UniqueCarrier      object 
 9   FlightNum          int64  
 10  TailNum            object 
 11  ActualElapsedTime  float64
 12  CRSElapsedTime     float64
 13  AirTime            float64
 14  ArrDelay           float64
 15  DepDelay           float64
 16  Origin             object 
 17  Dest               object 
 18  Distance           int64  
 19  TaxiIn             float64
 20  TaxiOut            float64
 21  Cancelled          int64  
 22  CancellationCode   object 
 23  Diverted           int64  
 24  CarrierDelay       float64
 25  WeatherDelay      

In [8]:
#elimino filas con NAN que hay en la columna ArrDelay 
dflimpio=df.dropna(subset=['ArrDelay'])

In [9]:
dflimpio.ArrDelay.isnull().sum()

0

In [10]:
dfl=dflimpio.copy()

In [11]:
#1 si hay  retraso 0 si ha llegado sin retraso
dfl['llegadas']= dfl['ArrDelay'].apply(lambda x: 1 if x >0 else 0)

In [12]:
dfl.llegadas.value_counts()

1    1723415
0     204956
Name: llegadas, dtype: int64

In [13]:
dfl.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,llegadas
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN,0
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN,1
2,2008,1,3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN,1
3,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0,1
4,2008,1,3,4,1940.0,1915,2121.0,2110,WN,378,N726SW,101.0,115.0,87.0,11.0,25.0,IND,JAX,688,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN,1


In [14]:
target_column = ['llegadas']

In [15]:
predictors=['DepDelay']

In [16]:
y = pd.DataFrame(dfl, columns=target_column)

In [17]:
X=pd.DataFrame(dfl, columns=predictors)

In [18]:
#divido DFs en trainset y test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state=42)

In [19]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1292008, 1), (636363, 1), (1292008, 1), (636363, 1))

Árbol de decisión

In [20]:
clfdf=DecisionTreeClassifier(class_weight='balanced')

In [21]:
clfdf.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced')

In [22]:
y_pred_clfdf = clfdf.predict(X_test)

Naive Bayes

In [23]:
clfGNB = GaussianNB()
clfGNB.fit(X_train, y_train.values.ravel())
y_predGNB = clfGNB.predict(X_test)

KNN NeighborsClassifier

In [24]:
neighbor = KNeighborsClassifier(n_neighbors=2)

In [25]:
neighbor.fit(X_train, y_train.values.ravel())

KNeighborsClassifier(n_neighbors=2)

In [26]:
y_predKNN=neighbor.predict(X_test)

# Exercici 2

Compara els models de classificació utilitzant la precisió (accuracy), una matriu de confiança i d’altres mètriques més avançades.

Accuracy

Árbol de decisión

In [27]:
print("Accuracy del Árbol de decisión:",metrics.accuracy_score(y_test, y_pred_clfdf))

Accuracy del Árbol de decisión: 0.7209925781354353


In [28]:
cm_clf = metrics.confusion_matrix(y_test, y_pred_clfdf,normalize='true')
print(cm_clf)

[[0.8945703  0.1054297 ]
 [0.29954064 0.70045936]]


In [29]:
print("La precision del Árbol de decisión : " ,metrics.precision_score(y_test, y_pred_clfdf))

La precision del Árbol de decisión :  0.9825064334519783


Naive Bayes

In [30]:
print("Accuracy del Naive Bayes:",metrics.accuracy_score(y_test, y_predGNB))

Accuracy del Naive Bayes: 0.7378728807300236


In [31]:
cm_clnb = metrics.confusion_matrix(y_test, y_predGNB,normalize='true')
print(cm_clnb)

[[0.87224244 0.12775756]
 [0.27802224 0.72197776]]


In [32]:
print("La precision de Naive Bayes : " ,metrics.precision_score(y_test, y_predGNB))

La precision de Naive Bayes :  0.9794964714857906


KNN

In [33]:
print("Accuracy del KNN:",metrics.accuracy_score(y_test, y_predKNN))

Accuracy del KNN: 0.8046602332316618


In [34]:
cm_knn = metrics.confusion_matrix(y_test, y_predKNN,normalize='true')
print(cm_knn)

[[0.35215034 0.64784966]
 [0.14181053 0.85818947]]


In [35]:
print("La precision de knn : " ,metrics.precision_score(y_test, y_predKNN))

La precision de knn :  0.9180203059998759


In [36]:
#Como se puede ver de los tres algoritmos el mejor resultado ha dado KNN NeighborsClassifier segun accuracy
#tabmien el que mas ha tardado. 

In [37]:
#El mas preciso es el arbol de decision

# Exercici 3

Entrena’ls utilitzant els diferents paràmetres que admeten.

In [38]:
#Árbol de decisión

In [50]:
clfdf2=DecisionTreeClassifier(class_weight='balanced',criterion='entropy',max_depth=3)

In [51]:
clfdf2.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=3)

In [52]:
y_pred_clfdf2 = clfdf2.predict(X_test)

In [53]:
print("Accuracy del Árbol de decisión:",metrics.accuracy_score(y_test, y_pred_clfdf2))

Accuracy del Árbol de decisión: 0.7378728807300236


In [54]:
print("La precision del Árbol de decisión : " ,metrics.precision_score(y_test, y_pred_clfdf2))

La precision del Árbol de decisión :  0.9794964714857906


KNN

In [55]:
neighbor2 = KNeighborsClassifier(n_neighbors=3, algorithm="ball_tree" )

In [56]:
neighbor2.fit(X_train, y_train.values.ravel())

KNeighborsClassifier(algorithm='ball_tree', n_neighbors=3)

In [57]:
y_predKNN2=neighbor2.predict(X_test)

In [58]:
print("La precision de knn : " ,metrics.precision_score(y_test, y_predKNN2))

La precision de knn :  0.9098812510114905


In [59]:
print("Accuracy del KNN:",metrics.accuracy_score(y_test, y_predKNN2))

Accuracy del KNN: 0.8699201556344414


# Exercici 4

Compara el seu rendiment utilitzant l’aproximació traint/test o cross-validation.

In [66]:
#KNN2
print(classification_report(y_test, y_predKNN2))

              precision    recall  f1-score   support

           0       0.32      0.21      0.25     67315
           1       0.91      0.95      0.93    569048

    accuracy                           0.87    636363
   macro avg       0.62      0.58      0.59    636363
weighted avg       0.85      0.87      0.86    636363



In [67]:
#KNN
print(classification_report(y_test, y_predKNN))

              precision    recall  f1-score   support

           0       0.23      0.35      0.28     67315
           1       0.92      0.86      0.89    569048

    accuracy                           0.80    636363
   macro avg       0.57      0.61      0.58    636363
weighted avg       0.84      0.80      0.82    636363



In [69]:
##Árbol de decisión 2

In [70]:
print(classification_report(y_test, y_pred_clfdf2))

              precision    recall  f1-score   support

           0       0.27      0.87      0.41     67315
           1       0.98      0.72      0.83    569048

    accuracy                           0.74    636363
   macro avg       0.63      0.80      0.62    636363
weighted avg       0.90      0.74      0.79    636363



In [71]:
##Árbol de decisión 1

In [72]:
print(classification_report(y_test, y_pred_clfdf))

              precision    recall  f1-score   support

           0       0.26      0.89      0.40     67315
           1       0.98      0.70      0.82    569048

    accuracy                           0.72    636363
   macro avg       0.62      0.80      0.61    636363
weighted avg       0.91      0.72      0.77    636363



## Nivell 2



# Exercici 5

Realitza algun procés d’enginyeria de variables per millorar-ne la predicció

In [280]:
dfl.head()

,level_0,index,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,llegadas,vel_media
0,0,0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN,0,419.0
1,1,1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN,1,430.1
2,2,2,2008,1,3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN,1,406.6
3,3,3,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0,1,401.3
4,4,4,2008,1,3,4,1940.0,1915,2121.0,2110,WN,378,N726SW,101.0,115.0,87.0,11.0,25.0,IND,JAX,688,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN,1,474.5


In [491]:
dfl2=dfl.copy()

In [519]:
#para no tener problemas de divison 0 eliminio los viajes que han durado 0

In [496]:
dfl2=dfl2[dfl2.AirTime>0]

In [500]:
dfl2["vel_media"]=dfl2['Distance']/(dfl2['AirTime']/60)

<ipython-input-500-9cdc02374885>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfl2["vel_media"]=dfl2['Distance']/(dfl2['AirTime']/60)


In [502]:
dfl2[['Distance','AirTime','vel_media']].describe().round(2)

,Distance,AirTime,vel_media
count,1928364.00,1928364.00,1928364.00
mean,764.95,108.28,396.95
std,573.89,68.64,97.51
min,11.00,1.00,21.30
25%,338.00,58.00,351.90
50%,606.00,90.00,403.80
75%,997.00,137.00,448.10
max,4962.00,1091.00,55920.00


In [503]:
target_column2 = ['llegadas']

In [504]:
predictors2=['DepDelay','vel_media']

In [509]:
y2 = pd.DataFrame(dfl2, columns=target_column2)

In [510]:
X2=pd.DataFrame(dfl2, columns=predictors2)

In [511]:
neighbor2 = KNeighborsClassifier(n_neighbors=9)

In [512]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.33,random_state=42)

In [513]:
clfdf3.fit(X_train2,y_train2)

DecisionTreeClassifier(class_weight='balanced')

In [514]:
y_pred_clfdf3 = clfdf3.predict(X_test2)

In [515]:
y_pred_clfdf3

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [516]:
print(classification_report(y_test2, y_pred_clfdf3))

              precision    recall  f1-score   support

           0       0.30      0.84      0.44     67242
           1       0.98      0.76      0.86    569119

    accuracy                           0.77    636361
   macro avg       0.64      0.80      0.65    636361
weighted avg       0.90      0.77      0.81    636361



In [517]:
#Parece que se ha mejorado un poco


# Nivell 3


# Exercici 6

No utilitzis la variable DepDelay a l’hora de fer prediccions

In [520]:
# Voy a usar las variables que tienen delay para predicir delay sin usar depdelay
#el rertaso total se compone de los retrasos parciales



In [574]:
dfl2.isnull().sum()

level_0              0
index                0
Year                 0
Month                0
DayofMonth           0
DayOfWeek            0
DepTime              0
CRSDepTime           0
ArrTime              0
CRSArrTime           0
UniqueCarrier        0
FlightNum            0
TailNum              0
ActualElapsedTime    0
CRSElapsedTime       0
AirTime              0
ArrDelay             0
DepDelay             0
Origin               0
Dest                 0
Distance             0
TaxiIn               0
TaxiOut              0
Cancelled            0
CancellationCode     0
Diverted             0
CarrierDelay         0
WeatherDelay         0
NASDelay             0
SecurityDelay        0
LateAircraftDelay    0
llegadas             0
vel_media            0
dtype: int64

In [576]:
dflimpio2['dftotal']=dflimpio2['CarrierDelay']+dflimpio2['WeatherDelay']+dflimpio2['NASDelay']+dflimpio2['SecurityDelay']+dflimpio2['LateAircraftDelay']

In [577]:
target_column1 = ['ArrDelay']
predictors1=['dftotal']
y3 = pd.DataFrame(dflimpio2, columns=target_column1)
X3 = pd.DataFrame(dflimpio2, columns=predictors1)



In [600]:
y3['ArrDelay']= y3['ArrDelay'].apply(lambda x: 1 if x >0 else 0)

In [601]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.33,random_state=42)

In [602]:
clfdf4=DecisionTreeClassifier(class_weight='balanced',criterion='entropy',max_depth=3)

In [603]:
clfdf4.fit(X_train3,y_train3)

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=3)

In [605]:
y_pred_clfdf4 = clfdf4.predict(X_test3)

In [606]:
print("Accuracy del KNN:",metrics.accuracy_score(y_test3, y_pred_clfdf4))

Accuracy del KNN: 1.0


In [607]:
print(classification_report(y_test3, y_pred_clfdf4))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00    411669

    accuracy                           1.00    411669
   macro avg       1.00      1.00      1.00    411669
weighted avg       1.00      1.00      1.00    411669



In [608]:
clfGNB2 = GaussianNB()
clfGNB2.fit(X_train3,y_train3.values.ravel())
y_predGNB2 = clfGNB2.predict(X_test3)

In [612]:
print(classification_report(y_test3, y_predGNB2))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00    411669

    accuracy                           1.00    411669
   macro avg       1.00      1.00      1.00    411669
weighted avg       1.00      1.00      1.00    411669



In [615]:
#100 % de aciertos. Seguramente hay overfitting